# Análisis Exploratorio de Datos (EDA)

Importación de librerías

In [1]:
import sqlite3 # Para manejo de bases de datos
import pandas as pd # Para manejo de dataframes

### Carga y exploración incial de datos

In [2]:
# Cargar datos
df = pd.read_csv("../data/raw/ufo_data.csv")
print("✅ Datos cargados")

FileNotFoundError: [Errno 2] No such file or directory: '../data/raw/ufo_data.csv'

In [17]:
# Crearemos una conexión sqlite3
conexion = sqlite3.connect("../data/raw/ufo_data.db")

# Guardar el dataframe como tabla SQLite
df.to_sql("ufo_table", conexion, if_exists="replace", index=False) 
# El index col se usa para eliminar el Unnamed

print("✅ Base de datos creada correctamente...")

cursor = conexion.cursor()

✅ Base de datos creada correctamente...


### Antes de empezar a trabajar, debemos crear una copia de la DB para no alterar a la original.

In [18]:
# Crear una copia de la db
copia = "../data/processed/copia.db"

consulta = f"""
ATTACH DATABASE '{copia}' AS copia;
SELECT sql FROM sqlite_master WHERE type='table';
"""
cursor.executescript(consulta)

# Copiar las tablas a la base de datos de respaldo
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tablas = cursor.fetchall()

for tabla in tablas:
    nombre_tabla = tabla[0]
    consulta_copia = f"CREATE TABLE copia.{nombre_tabla} AS SELECT * FROM {nombre_tabla};"
    cursor.execute(consulta_copia)

print("✅ Copia de la BD creada correctamente...")
# Ya no volver a ejecutar esto
# Si lo hacemos primero hay que cerrar la conexión
# Y luego hay que eliminar el archivo copia.db

OperationalError: table ufo_table already exists

### Consultar todas las tablas que existen en el Dataset o DB

In [10]:
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())

[('ufo_table',)]


### Explorar las 5 principales filas del Dataset(DB)

In [6]:
filtrar_5_filas = "SELECT * FROM copia.ufo_table LIMIT 5;"
filtrar_5_filas = pd.read_sql_query(filtrar_5_filas, conexion)
filtrar_5_filas

,datetime,city,state,country,shape,duration (seconds),duration (hours/min),comments,date posted,latitude,longitude
0,1949-10-10 20:30:00,san marcos,tx,us,cylinder,2700,45 minutes,This event took place in early fall around 194...,2004-04-27,29.8830556,-97.941111
1,1949-10-10 21:00:00,lackland afb,tx,None,light,7200,1-2 hrs,1949 Lackland AFB&#44 TX. Lights racing acros...,2005-12-16,29.38421,-98.581082
2,1955-10-10 17:00:00,chester (uk/england),None,gb,circle,20,20 seconds,Green/Orange circular disc over Chester&#44 En...,2008-01-21,53.2,-2.916667
3,1956-10-10 21:00:00,edna,tx,us,circle,20,1/2 hour,My older brother and twin sister were leaving ...,2004-01-17,28.9783333,-96.645833
4,1960-10-10 20:00:00,kaneohe,hi,us,light,900,15 minutes,AS a Marine 1st Lt. flying an FJ4B fighter/att...,2004-01-22,21.4180556,-157.803611


### Obtener información sobre la estructura de la tabla

In [13]:
info = "PRAGMA table_info(ufo_table);"
info = pd.read_sql_query(info, conexion)
info

,cid,name,type,notnull,dflt_value,pk
0,0,datetime,TEXT,0,None,0
1,1,city,TEXT,0,None,0
2,2,state,TEXT,0,None,0
3,3,country,TEXT,0,None,0
4,4,shape,TEXT,0,None,0
5,5,duration (seconds),TEXT,0,None,0
6,6,duration (hours/min),TEXT,0,None,0
7,7,comments,TEXT,0,None,0
8,8,date posted,TEXT,0,None,0
9,9,latitude,TEXT,0,None,0


### Estadísticas Descriptivas

#### Total de registros o filas

In [14]:
total_registros = "SELECT COUNT(*) AS total_registros FROM copia.ufo_table;"
total_registros = pd.read_sql_query(total_registros, conexion)
total_registros

,total_registros
0,80332


In [ ]:
# 

1. Identificar o explorar:

    ✅ Datos faltantes (Nulos).
    
    ✅ Registros duplicados.
    
    ✅ Formatos inconsistentes (fechas, nombres, números).
    
    ✅ Valores atípicos (outliers).

In [ ]:
# Cerrar conexión una vez terminado
#conexion.close()